<a href="https://colab.research.google.com/github/yes-yay/CD-codes/blob/main/generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#sri rama jayam
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
with open('TinyStories-train.txt','r' , encoding='utf-8') as f:
  text=f.read()

In [ ]:
len(text)

809912972

In [ ]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)



	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£§«­°´»¿ÂÉàáâéíïñóöāі   ​‎–—―‘’“”„…  ‪′−─」ﬁ﻿�𝑐🌴🌹🍌🙂
152


In [ ]:
stoi= {ch:i for i,ch in enumerate(chars)}
itos= {i:ch for i,ch in enumerate(chars)}

encode=lambda s: [stoi[c] for c in s]
decode=lambda l: ''.join([itos[i] for i in l])

print(encode("hi macha"))
print(decode(encode("hi macha")))

[73, 74, 2, 78, 66, 68, 73, 66]
hi macha


In [ ]:
text=torch.tensor(encode(text) , dtype=torch.long)

In [ ]:
n=int(0.9 * len(text))
train_data=text[:n]
val_data=text[n:]


In [ ]:
block_size=8
train_data[:block_size+1]

In [ ]:
torch.manual_seed(1337)
batch_size=4
block_size=8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

In [ ]:
#self attention
torch.manual_seed(1337)
B,T,C = 4,8,32
x=torch.randn(B,T,C)
print(x.shape)

head_size=16
key=nn.Linear(C , head_size , bias=False)
query=nn.Linear(C , head_size , bias=False)
value=nn.Linear(C , head_size, bias=False)

k=key(x) #(B,T,head_size)
q=query(x) # same here also



wei=q @ k.transpose(-2,-1) * head_size ** -0.5 # (B , T , 16) @ (B , 16 , T) = (B, T , T)

wei=F.softmax(wei , dim=1)
print(wei.shape)

v=value(x)

out=wei @ v
out.shape


In [ ]:
batch_size = 64
block_size = 256
n_embed = 256
n_heads = 8
n_layers = 6
dropout = 0.2
learning_rate = 3e-4
max_iters = 2000


In [ ]:
class Head(nn.Module):

  def __init__(self , head_size):
    super().__init__()
    self.key=nn.Linear(n_embed , head_size , bias=False)
    self.query=nn.Linear(n_embed , head_size , bias=False)
    self.value=nn.Linear(n_embed , head_size , bias=False)
    self.register_buffer('tril' , torch.tril(torch.ones(block_size , block_size)))

  def forward(self , x):
    B,T,C=x.shape
    k=self.key(x)
    q=self.query(x)
    v=self.value(x)

    wei= q @ k.transpose(-2,-1) * C ** -0.5
    wei=wei.masked_fill(self.tril[:T,:T]==0 , float('-inf'))
    wei=F.softmax(wei , dim=-1)

    out=wei @ v
    return out


In [ ]:
head_1 = torch.tensor([[[1, 2], [3, 4], [5, 6]]])  # Shape: (1, 3, 2)

head_2 = torch.tensor([[[7, 8], [9, 10], [11, 12]]])  # Shape: (1, 3, 2)

head_1.shape

print(torch.cat((head_1 , head_2 ), dim=-1))
print(torch.cat((head_1 , head_2) , dim=-2))

tensor([[[ 1,  2,  7,  8],
         [ 3,  4,  9, 10],
         [ 5,  6, 11, 12]]])
tensor([[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10],
         [11, 12]]])


In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self , num_heads , head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(n_embed,n_embed)
    self.dropout=nn.Dropout(dropout)

  def forward(self , x):
    out=torch.cat([h(x) for h in self.heads] , dim=-1)
    out=self.dropout(self.proj(out))
    return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
class Block(nn.Module):
    def __init__(self, n_embed, num_heads):
        super().__init__()
        head_size = n_embed // num_heads
        self.sa = MultiHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embed)
        self.position_embedding = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_heads) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx)
        pos_emb = self.position_embedding(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            logits = logits.view(-1, vocab_size)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = BigramLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for step in range(max_iters):
    xb, yb = get_batch("train")
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 250 == 0:
        print(f"Step {step}: Loss = {loss.item()}")

Step 0: Loss = 1.187935471534729
Step 250: Loss = 1.128569483757019
Step 500: Loss = 1.1200956106185913
Step 750: Loss = 1.0485488176345825
Step 1000: Loss = 1.0203546285629272
Step 1250: Loss = 1.060569405555725
Step 1500: Loss = 1.083422064781189
Step 1750: Loss = 0.974388599395752


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long)

context=context.to(device)

# Generate 100 new tokens
generated_indices = model.generate(context, max_new_tokens=100)

# Decode it to text
generated_text = decode(generated_indices[0].tolist())
print(generated_text)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
